## Reshaping Data for Visualization

In [1]:
# To list files in a directory
import os 

# For data manipulation
import pandas as pd
import numpy as np

from lets_plot import *
LetsPlot.setup_html()

In [2]:
# List all files in the ME204/data/waitrose folder
all_files = [os.path.join('../data/waitrose', file) for file in os.listdir('../data/waitrose') 
             if file.endswith('.csv')]

# Read every single file and concatenate them into a single DataFrame with pandas concat
df = pd.concat((pd.read_csv(file) for file in all_files))

In [3]:
df

,data-product-id,data-product-name,data-product-type,data-product-on-offer,data-product-index,image-url,product-page,product-name,product-size,item-price,price-per-unit,offer-description,category
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,£3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,£3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,£13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,95p,1.2p each,NaN,"Baby, Child & Parent"
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,£1.60,£4/100g,NaN,"Baby, Child & Parent"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,18427,Duchy Organic British Lamb Half Leg,G,False,1590.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic British Lamb Half Leg,Typical weight 0.94kg,£24.31 each est.,£26.00/kg,NaN,Waitrose Brands
1590,11606,Cooks' Ingredients Wok Oil,G,False,1591.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients Wok Oil,250ml,£2.40,96p/100ml,NaN,Waitrose Brands
1591,6903,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,G,False,1592.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no-1-ba...,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,350g,£10.00,£28.58/kg,NaN,Waitrose Brands
1592,6125,Cooks' Ingredients White Marzipan,G,False,1593.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients White Marzipan,500g,£2.50,£5/kg,NaN,Waitrose Brands


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 2.5+ MB


In [5]:
# Drop duplicates
df = df.drop_duplicates()

df = df.drop(columns=['data-product-name', 
                      'data-product-type', 
                      'data-product-index'])
df = (
    df.rename(columns={
        'data-product-id': 'id',
        'data-product-on-offer': 'offer',
        'product-page': 'page',
        'product-name': 'name',
        'product-size': 'size',
    })
)

# The id does not need 64 bits. 32 bits is enough.
# See https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.intc for ranges.
df['id'] = df['id'].astype('int32')

In [6]:
def clean_item_price(item_price: str):    
    """
    Cleans the item price string by performing necessary transformations.

    Parameters:
    item_price (str): The item price as a string.

    Returns:
    str: The cleaned item price.
    """
    if ' each est.' in item_price:
        item_price = item_price.replace(' each est.','')

    if type(item_price) != str:
        pass
    
    elif '£' in item_price:
        item_price = item_price.replace('£', '')
    elif 'p' in item_price:
        item_price = item_price.replace('p', '')
        item_price = '0.' + item_price
    if'-' in item_price:
        item_price=item_price.split('-')[0]
        
    return float(item_price)

In [7]:
df['item-price-fixed']=df['item-price'].astype(str).apply(clean_item_price)
df[['item-price-fixed','item-price']]

,item-price-fixed,item-price
0,3.15,£3.15
1,3.15,£3.15
2,13.50,£13.50
3,0.95,95p
4,1.60,£1.60
...,...,...
1589,24.31,£24.31 each est.
1590,2.40,£2.40
1591,10.00,£10.00
1592,2.50,£2.50


In [8]:
selected_rows= df['item-price'].astype(str).str.contains('-')
df.loc[selected_rows,'size']=df.loc[selected_rows,'size'].str.split('-').str[0].str.replace('(','')

In [9]:
df[selected_rows]


,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category,item-price-fixed
291,724969,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no1-bri...,No.1 British Venison Rack,1kg,£55-£110 each est.,£55 / kg,NaN,Best of British,55.0
294,676707,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/carvery...,Carvery Leg of British Lamb Studded with Garli...,1.4kg,£30-£38 each est.,£21 / kg,NaN,Best of British,30.0
299,646311,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no1-fiv...,No.1 Five Bone French-Trimmed British Veal Rib,2kg,£54-£92 each est.,£27 / kg,NaN,Best of British,54.0
302,574495,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no1-30-...,No.1 30 Day Dry Aged Jubilee Rib of British Beef,6kg,£168-£196 each est.,£28 / kg,NaN,Best of British,168.0
305,544362,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/british...,British Native Breed Whole Beef Brisket,2.2kg,£31-£40 each est.,£14 / kg,NaN,Best of British,31.0
307,513226,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/british...,British Native Breed Beef Fillet,1.8kg,£94-£115 each est.,£52 / kg,NaN,Best of British,94.0
311,444210,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no1-fre...,No.1 Free Range British Pork Hog Roast with Sa...,3.1kg,£35-£41 each est.,£11 / kg,NaN,Best of British,35.0
313,415568,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/british...,British Lamb Guard of Honour with a Pesto & Pi...,0.9kg,£32-£39 each est.,£35 / kg,NaN,Best of British,32.0
318,79552,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/boneles...,Boneless Leg of British Lamb with Apricot & Ro...,1.1kg,£24-£32 each est.,£21 / kg,NaN,Best of British,24.0
3237,840603,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no1-dry...,No.1 Dry Aged Bone in Sirloin of British Beef,1.2kg,£42-£60 each est.,£35 / kg,NaN,Fresh & Chilled,42.0


In [10]:
df['item-price']=df['item-price-fixed']
df.drop(columns=['item-price-fixed'],inplace=True)


In [11]:
df[selected_rows][['name','size','item-price']].head()

,name,size,item-price
291,No.1 British Venison Rack,1kg,55.0
294,Carvery Leg of British Lamb Studded with Garli...,1.4kg,30.0
299,No.1 Five Bone French-Trimmed British Veal Rib,2kg,54.0
302,No.1 30 Day Dry Aged Jubilee Rib of British Beef,6kg,168.0
305,British Native Breed Whole Beef Brisket,2.2kg,31.0


### Exercise 02: What is the distribution of prices per category?


In [12]:
plot_df=df.groupby('category')['item-price'].describe().reset_index()
plot_df.head()

,category,count,mean,std,min,25%,50%,75%,max
0,"Baby, Child & Parent",428.0,5.196215,4.677661,0.6,1.8000,3.075,7.2625,25.00
1,Bakery,482.0,4.846680,7.750208,0.5,1.6000,2.200,3.1500,45.00
2,"Beer, Wine & Spirits",1679.0,14.425057,15.720801,0.3,5.3700,9.990,17.9900,299.99
3,Best of British,322.0,6.928106,12.570727,0.1,2.7125,4.150,6.0000,168.00
4,Dietary & Lifestyle,3337.0,3.795999,4.755322,0.1,1.9500,2.750,4.0000,139.99


In [13]:
plot_df=plot_df.rename(columns={
    '50%':'median',
    '25%':'Q1',
    '75%':'Q3'})
plot_df

,category,count,mean,std,min,Q1,median,Q3,max
0,"Baby, Child & Parent",428.0,5.196215,4.677661,0.60,1.8000,3.075,7.2625,25.00
1,Bakery,482.0,4.846680,7.750208,0.50,1.6000,2.200,3.1500,45.00
2,"Beer, Wine & Spirits",1679.0,14.425057,15.720801,0.30,5.3700,9.990,17.9900,299.99
3,Best of British,322.0,6.928106,12.570727,0.10,2.7125,4.150,6.0000,168.00
4,Dietary & Lifestyle,3337.0,3.795999,4.755322,0.10,1.9500,2.750,4.0000,139.99
5,Everyday Value,141.0,1.740284,1.202873,0.15,1.1000,1.350,2.0000,8.00
6,Food Cupboard,4186.0,2.665160,2.138691,0.35,1.6000,2.250,3.0000,43.50
7,Fresh & Chilled,3578.0,3.815151,5.267549,0.10,2.0000,2.900,4.1425,168.00
8,Frozen,431.0,3.627378,1.544523,0.70,2.6500,3.500,4.4750,12.00
9,Home,1075.0,8.595033,10.764164,0.49,3.0000,5.990,10.5000,159.00


In [14]:
# This configures what shows up when you hover your mouse over the plot.
tooltip_setup = (
    layer_tooltips()
        .line('@category')
        .line('[@Q1 , @median , @Q3]')
        .format('@Q1', '£ {.2f}')
        .format('@median', '£ {.2f}')
        .format('@Q3', '£ {.2f}')
)

g = (
    # Maps the columns to the aesthetics of the plot.
    ggplot(plot_df, aes(y='category', x='median', xmin='Q1', xmax='Q3', fill='category')) +

    # GEOMS

    # Add a line range that 'listens to' columns informed in `ymin` and `ymax` aesthetics
    geom_linerange(size=1, alpha=0.75, tooltips=tooltip_setup) +

    # Add points to the plot (listen to `x` and `y` and fill aesthetics)
    geom_point(size=3, stroke=1, shape=21, tooltips=tooltip_setup) +

    # SCALES

    # Remove the legend (we can already read the categories from the y-axis)
    scale_fill_discrete(guide='none') +

    # Specify names for the axes
    scale_y_continuous(name="Categories\n(from largest to smallest median)", expand=[0.05, 0.05]) +
    scale_x_continuous(name="Price (£)", expand=[0., 0.05], format='£ {.2f}', breaks=np.arange(0, 20, 2.5)) +

    # LABELS
    # It's nice when the plot tells you the key takeaways
    labs(title='"Beer, Wine & Spirits" has the highest median price for individual items',
         subtitle="Dots represent the median price, bars represent the 25th and 75th percentiles") +
    theme(axis_text_x=element_text(size=15),
        axis_text_y=element_text(size=17),
        axis_title_x=element_text(size=20),
        axis_title_y=element_text(size=20),
        plot_title=element_text(size=19, face='bold'),
        plot_subtitle=element_text(size=18),
        legend_position='none') +
    ggsize(1000, 500)

)

g

### Exercise 03: Are there any products that appear in multiple categories?

In [15]:
duplicates_df=df.groupby('name')['category'].unique().reset_index()
duplicates_df

,name,category
0,019521 - Essential Flageolet Beans in Water,"[Dietary & Lifestyle, Food Cupboard, Waitrose ..."
1,035738 - Solero Exotic Ice cream Lolly,"[Dietary & Lifestyle, Frozen, Summer]"
2,044889 - Essential Double Cream Large,"[Fresh & Chilled, Waitrose Brands]"
3,060167 - Kronenbourg 1664 Cans,"[Beer, Wine & Spirits]"
4,086143 - Cooks' Ingredients Garlic,"[Dietary & Lifestyle, Fresh & Chilled, Waitros..."
...,...,...
15413,"£100,000 a Month for a Year Scratchcard",[Home]
15414,Öpso Japanese Sakura Tree Reed Diffuser,"[Home, Household]"
15415,Öpso Mediterranean Citrus Grove Reed Diffuser,"[Home, Household]"
15416,Öpso Nordic Birch Forest Reed Diffuser,"[Home, Household]"


In [16]:
duplicates_df['counts']=df.groupby('name')['category'].nunique().values
duplicates_df.sort_values('counts',ascending=False)

,name,category,counts
3512,Duchy Organic Houmous,"[Dietary & Lifestyle, Fresh & Chilled, New, Or...",6
4678,Essential Sweetcorn,"[Dietary & Lifestyle, Everyday Value, Food Cup...",6
3608,Duchy Organic Unhomogenised Whole Milk 4 Pints,"[Best of British, Dietary & Lifestyle, Fresh &...",6
4404,Essential Limes,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev...",6
4234,Essential Cucumber,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev...",6
...,...,...,...
15388,itsu Rice Cakes Milk Chocolate,[Food Cupboard],1
15390,itsu Salted Caramel Bao Buns,[Dietary & Lifestyle],1
15391,itsu Satay Rice Noodles,[Food Cupboard],1
15376,itsu Dark Chocolate Rice Cakes,[Food Cupboard],1


In [17]:
duplicates_df=duplicates_df[duplicates_df['counts']>1]
duplicates_df.sort_values('counts')

,name,category,counts
64,33cm Floral Oasis Napkins 20s,"[Home, Summer]",2
12309,Tri-Molecular Hyaluronic Serum,"[Dietary & Lifestyle, Toiletries, Health & Bea...",2
2,044889 - Essential Double Cream Large,"[Fresh & Chilled, Waitrose Brands]",2
12313,Trimmed Mange Tout Peas,"[Dietary & Lifestyle, Fresh & Chilled]",2
5,10 Mins to Bed Where's Little Dinosaur,"[Home, New]",2
...,...,...,...
3512,Duchy Organic Houmous,"[Dietary & Lifestyle, Fresh & Chilled, New, Or...",6
3608,Duchy Organic Unhomogenised Whole Milk 4 Pints,"[Best of British, Dietary & Lifestyle, Fresh &...",6
4404,Essential Limes,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev...",6
4678,Essential Sweetcorn,"[Dietary & Lifestyle, Everyday Value, Food Cup...",6


In [18]:
df[df.duplicated('id',keep='first')].count()

id                   9323
offer                9323
image-url            9323
page                 9323
name                 9317
size                 9299
item-price           9317
price-per-unit       9227
offer-description    2522
category             9323
dtype: int64